In [1]:
#Caricare le librerie
import tweepy
import pandas as pd
from tqdm import tqdm
import re

In [2]:
consumer_key= "rrR0Znz98oKkjfB8b72oLH9Qc"
consumer_secret= "m7iOPX6iW8R3IalRf7x56YVeIb18o3I1bhUftg8JzlQ6fAjLgW"
access_token= "1442250607-3cJYKYyh7Ae19nMcv5YoxOIB2XlvgVX0ROaDSJx"
access_token_secret= "ZOnaaCTgO7DsKtmPjVruOlkvzBY8p6MbkKLjiYLdamBYE"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [13]:
#Scaricare in un dataframe i tweet in base ad un hashtag (until="2021-12-21")
#NB: hasattr(obj, name, /) Return whether the object has an attribute with the given name.

hashtags=["#novax","#vaccinoobbligatorio"]
list_tweet=[]
list_originals=[]
list_rt_id=[]
list_retweetted=[]


for hashtag in hashtags:
  next_cursor=None
  n=5000
  pbar = tqdm(total=n)
  while n>0:
    for tweet in tweepy.Cursor(api.search_tweets,q=hashtag,lang="it",cursor=next_cursor).items(n):

      #gestione dei retweet con try-except
      try:
        list_originals.append(tweet.retweeted_status.id) #creo lista dei retweets id 
        list_rt_id.append(tweet.retweeted_status.user.id)

        #lista tweets che sono stati retweettati
        try: 
          rt_text=tweet.retweeted_status.extended_tweet["full_text"] 
        except AttributeError:
          rt_text=tweet.retweeted_status.text

        list_retweetted.append([tweet.retweeted_status.created_at, tweet.retweeted_status.id,tweet.retweeted_status.user.id, rt_text, 
                         tweet.retweeted_status.favorite_count, tweet.retweeted_status.retweet_count, 
                         tweet.retweeted_status.user.screen_name]) 
      except:
        list_originals.append(0)
        list_rt_id.append(0) 

      try: 
        tweet_text=tweet.extended_tweet["full_text"]
      except AttributeError:
        tweet_text=tweet.text

      list_tweet.append([tweet.created_at, tweet.id,tweet.user.id, tweet_text, 
                         tweet.favorite_count, tweet.retweet_count, 
                         tweet.user.location,tweet.user.screen_name,
                         hasattr(tweet, "retweeted_status"),
                         tweet.in_reply_to_screen_name])
      n -=1
      next_cursor=tweet
      pbar.update(n=1)

100%|██████████| 10000/10000 [12:25:57<00:00,  4.48s/it]
Rate limit reached. Sleeping for: 778
100%|██████████| 5000/5000 [29:58<00:00, 21.51it/s]  

In [17]:
#creo df dei tweet
tweets = pd.DataFrame(list_tweet, 
                      columns=['date','id','user_id','text','like','n_rt','location','author',
                               "retweet","reply_name"])

#arricchimento df tweets
tags_column=[]
weigth_column=[]
for text in tweets.text:
    tags_list=[]
    for tag in hashtags:
        finder=re.findall(tag,'in questa stringa è presente un #novax da #NoGreenPass trovare')
        if finder != []:
            tags_list.append(finder[0])

    if tags_list==['#novax']:
        weigth_column.append(2.0)
    elif tags_list==["#vaccinoobbligatorio"]:
        weigth_column.append(3.0)
    else:
        weigth_column.append(1.0)

    tags_column.append(tags_list)
    
tweets['hashtags']=tags_column
tweets['weights']=weigth_column
tweets["id_retweet"]=list_originals
tweets["user_id_retweet"]=list_rt_id


#creo df dei retweet
retweetted = pd.DataFrame(list_retweetted, 
                      columns=['date','id','author_id','text','like','n_rt','author'])                                                    
tweets.head()

,date,id,user_id,text,like,n_rt,location,author,retweet,reply_name,hashtags,weights,id_retweet,user_id_retweet
0,2021-12-27 11:18:16+00:00,1475425795413889035,2918340939,"Un #messaggio per i #novax: ""Ti devi volere pi...",0,0,,cinzianicolosi1,False,None,[#novax],2.0,0,0
1,2021-12-27 11:18:10+00:00,1475425771103653889,390915227,RT @FranAltomare: Col lockdown selettivo dei #...,0,130,,Arwine94,True,None,[#novax],2.0,1475373992320114696,505083
2,2021-12-27 11:17:38+00:00,1475425635766046721,1333921772,"Direi che per la piega che stanno prendendo, i...",0,0,Somewhere in Emilia,magnagnagne,False,None,[#novax],2.0,0,0
3,2021-12-27 11:17:31+00:00,1475425608498913280,291706540,RT @Giorgiolaporta: Con i tamponi ai vaccinati...,0,130,Italia,RaffaeleFor,True,None,[#novax],2.0,1475406344945606662,16145985
4,2021-12-27 11:17:30+00:00,1475425604665290756,1472111659,Sarebbe sufficiente seguire il modello #German...,0,0,Cagliari ~ Sardinia ~ Italy ☀️,tinaloi15,False,None,[#novax],2.0,0,0


In [18]:
tweets.shape

(10000, 14)

In [19]:
#selezione dei tweet più retwettati 
retweetted.sort_values("n_rt",ascending=False).drop_duplicates()

,date,id,author_id,text,like,n_rt,author
1042,2021-09-06 20:26:42+00:00,1434976371336433667,420931672,"#Cacciari: ""È l'ultimo atto di una vicenda, or...",1936,797,EmeiMarkus
1083,2021-12-20 11:35:57+00:00,1472893531769430019,112832545,- Mi racconti una barzelletta?\n- Tre dosi per...,2466,627,_GrayDorian
1817,2021-12-20 11:35:57+00:00,1472893531769430019,112832545,- Mi racconti una barzelletta?\n- Tre dosi per...,2467,627,_GrayDorian
1020,2021-12-09 19:45:15+00:00,1469030399368118273,16145985,Domandina semplice semplice: il #covid a #LaSc...,1383,534,Giorgiolaporta
2408,2021-12-15 12:03:10+00:00,1471088440259362818,420931672,Dott. Giovanni #Frajese - medico endocrinologo...,900,526,EmeiMarkus
...,...,...,...,...,...,...,...
5003,2021-12-21 18:42:17+00:00,1473363210136862721,1466721471791775745,"Ci cancellano i messaggi ma sui muri, come nel...",1,1,NellaBollano
3151,2021-12-26 09:53:51+00:00,1475042165931843590,823975885963399168,Qui ormai c'è gente che va a farsi un #tampone...,7,1,GiovyDean
3152,2021-12-26 12:09:52+00:00,1475076392832450566,595306303,L'Usl chiude le porte del Parini ai malati con...,0,1,CriSStraits
3364,2021-12-24 17:43:46+00:00,1474435645477203969,1091380531,"@ImolaOggi Lo spieghi al suo amico Silvio, che...",2,1,ansia_tw


##### Vado a vedere quali sono i tweet più retwittati e quante volte sono stati ritwittati dalla nostra rete (usando SQL)

In [20]:
#Open the terminal and run-   pip install U pandasql
#Open your Jupyter Notebook and in any cell run-    !pip install -U pandasql
import pandasql as ps
q1 = """SELECT id,like,n_rt, author,count() FROM retweetted group by id order by n_rt desc"""
print(ps.sqldf(q1, locals()))

                      id  like  n_rt           author  count()
0    1434976371336433667  1936   797       EmeiMarkus        4
1    1472893531769430019  2466   627      _GrayDorian       10
2    1469030399368118273  1383   534   Giorgiolaporta        1
3    1471088440259362818   900   526       EmeiMarkus        1
4    1472301392467476489  1498   510  strange_days_82        3
..                   ...   ...   ...              ...      ...
908  1468468995493924873     1     1  Libero_Giorgio1        1
909  1467093215627124736     3     1  pecorefannobooh        1
910  1464657492726714368     2     1     moenia_mundi        1
911  1442601843079647238     2     1   NoName06430673        1
912  1473457814416896005     3     0        TiTTiTaLe        1

[913 rows x 5 columns]


#### Creazione input Grephi tramite scrittura file .csv
Da rivedere:
- Abbiamo creato nodi con l'id del tweet, non per utente
- Si possono pesare gli archi tra utenti per il numero di retweet tra essi? Ha senso?

In [32]:
rt_tweets=tweets[tweets.retweet==True][['user_id','user_id_retweet','author','weights']]

In [33]:
tweets_ids=rt_tweets[['user_id','author']]
retwitted_ids=retweetted[['author_id','author']].rename(columns={'author_id':'user_id'})
rt_tweets_ids=pd.concat([tweets_ids,retwitted_ids]).drop_duplicates().reset_index(drop=True)

In [34]:
Nodes_df=pd.DataFrame({'Id':rt_tweets_ids.user_id,'Label':rt_tweets_ids.author,'Interval':None})

In [35]:
Edges_df=pd.DataFrame({'Source':rt_tweets.user_id,'Target':rt_tweets.user_id_retweet,'Type':'Directed','Kind':None,'Id':range(1, len(rt_tweets)+1),'Label':None,'Interval':None,'Weight':rt_tweets.weights})

In [36]:
Nodes_df.to_csv('Nodes_df2.csv', index=False)
Edges_df.to_csv('Edges_df2.csv', index=False)

##### Grafi con NetworkX

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
Edges_df=pd.read_csv("C:/Users/dvd_m/Desktop/Social Media Analytics/SMA-Project/Edges_df.csv")
G = nx.from_pandas_edgelist(Edges_df, source='Source', target='Target')
df_cluster=pd.DataFrame({"nodes":nx.clustering(G).keys(),"cluster":nx.clustering(G).values()})
grafo=pd.merge(Edges_df,df_cluster, left_on="Source",right_on="nodes")